In [ ]:
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')

from neurochat.nc_data import NData
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
import neurochat.nc_plot as nc_plot
from neurochat.nc_lfp import NLfp
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import math
import pynapple as nap
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import sys
import nwb4fp.analyses.maps as mapp
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap_ax,plot_path
from nwb4fp.analyses.fields import separate_fields_by_laplace, separate_fields_by_dilation,find_peaks,separate_fields_by_laplace_of_gaussian,calculate_field_centers,distance_to_edge_function, remove_fields_by_area, map_pass_to_unit_circle,which_field,compute_crossings
from elephant.statistics import time_histogram, instantaneous_rate
from nwb4fp.analyses import maps
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,Speed_filtered_spikes
from nwb4fp.data.helpers import unit_location_ch
from scipy.ndimage import gaussian_filter
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import numpy as np
from scipy.ndimage import gaussian_filter1d
from nwb4fp.analyses.phase_precession import cl_corr
import numpy as np
from scipy import signal
from scipy.signal import hilbert
import matplotlib.pyplot as plt
import nwb4fp.analyses.maps as mapp
import nwb4fp.analyses.maps as mapp

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)  # 前后向滤波，避免相位移
    return y

import numpy as np
from scipy import signal
import matplotlib.pyplot as plt




def plot_psd(freqs, psd, theta_range=(6, 12)):
    """
    Plot the power spectral density with theta range highlighted.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(freqs, psd, 'b-', label='PSD')
    
    # Highlight theta range
    plt.axvspan(theta_range[0], theta_range[1], color='gray', alpha=0.2, label='Theta range (6-12 Hz)')
    
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density')
    plt.title('Power Spectral Density of Spike Train Autocorrelogram')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 50)  # Limit to 50 Hz for better visualization of theta range
    plt.show()
    
def ThetaModulationIndex(spikes):
    # Convert spikes to milliseconds
    spikes_ms = spikes * 1e3
    binSize_ms = 5
    
    # Initialize array for mean sums
    sumsMean = np.zeros(100)
    
    # Process each spike
    for i in range(len(spikes_ms)):
        curSpike = spikes_ms[i]
        low_edge = curSpike
        upper_edge = curSpike + 500
        # Create bin edges
        edges = np.arange(low_edge, upper_edge + binSize_ms, binSize_ms)
        # Count spikes in bins
        N = np.histogram(spikes_ms, bins=edges)[0]
        # Subtract 1 from first bin to not count the current spike
        N[0] = N[0] - 1
        # Add to running sum
        sumsMean = sumsMean + N
    
    # Define bin ranges (converting time to bin indices)
    throughBins = [int(50/binSize_ms) + 1, int(70/binSize_ms) + 1]
    peakBins = [int(100/binSize_ms) + 1, int(140/binSize_ms) + 1]
    
    # Calculate means for through and peak periods
    through = np.mean(sumsMean[throughBins[0]-1:throughBins[1]])
    peak = np.mean(sumsMean[peakBins[0]-1:peakBins[1]])
    
    # Calculate theta modulation index
    thIndex = (peak- through) / (through + peak)
    
    return thIndex

# Example usage:
# spikes = np.array([0.1, 0.2, 0.3, 0.4])  # example spike times in seconds
# result = ThetaModulationIndex(spikes)
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def calculate_spike_theta_modulation(spike_times, fs=1000, freq_range=(0, 125), theta_range=(6, 12)):
    """
    Calculate theta modulation index for a spike train and return PSD and ACG for plotting.
    Excludes zero-lag from autocorrelation.
    
    Parameters:
    -----------
    spike_times : array-like
        Array of spike timestamps in seconds
    fs : float
        Sampling frequency in Hz (default: 1000 Hz)
    freq_range : tuple
        Frequency range for total power calculation (default: 0-125 Hz)
    theta_range : tuple
        Frequency range for theta band (default: 6-12 Hz)
        
    Returns:
    --------
    bool : Whether the spike train is theta modulated (index > 5)
    float : Theta modulation index
    array : Frequencies from PSD
    array : Power spectral density values
    array : Time lags for ACG
    array : Autocorrelogram values
    """
    #spike_train=spikes_time
    # Convert spike times to binary spike train
    try:
        duration = spike_times[-1] - spike_times[0]
        n_samples = int(duration * fs) + 1
        spike_train = np.zeros(n_samples)
        spike_indices = ((spike_times - spike_times[0]) * fs).astype(int)
        spike_train[spike_indices] = 1
        
        ##Calculate autocorrelogram (full mode)
        autocorr = signal.correlate(spike_train, spike_train, mode='full')
        
        # Exclude the zero-lag point (center of the autocorrelation)
        center = len(autocorr) // 2
        autocorr[center] = 0  # Set the zero-lag to 0
        autocorr = gaussian_filter1d(autocorr, sigma=2)
        # Take only positive lags for PSD calculation
        autocorr_psd = autocorr[center + 1:]  # Start from t=1
        
        # Calculate power spectral density
        freqs, psd = signal.welch(autocorr_psd, fs=fs, nperseg=min(1024, len(autocorr_psd)))
        
        # Find indices for frequency ranges
        freq_mask_total = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
        freq_mask_theta = (freqs >= theta_range[0]) & (freqs <= theta_range[1])
        
        # Find peak theta frequency
        theta_psd = psd[freq_mask_theta]
        theta_freqs = freqs[freq_mask_theta]
        peak_theta_idx = np.argmax(theta_psd)
        peak_theta_freq = theta_freqs[peak_theta_idx]
        
        # Calculate power in 2 Hz window around peak theta frequency
        theta_window_mask = (freqs >= (peak_theta_freq - 1)) & (freqs <= (peak_theta_freq + 1))
        
        # Calculate power ratios
        total_power = np.sum(psd[freq_mask_total])
        theta_power = np.sum(psd[theta_window_mask])
        
        # Calculate modulation index
        modulation_index = (theta_power / total_power) * 100
        
        # Check if theta modulated (index > 5)
        is_theta_modulated = modulation_index > 5
        
        # Prepare ACG for plotting (full range with positive and negative lags)
        autocorr_full = signal.correlate(spike_train, spike_train, mode='full')
        autocorr_full[center] = 0  # Exclude zero-lag
        lags = np.arange(-center, center + 1) / fs  # Time lags in seconds
        
        return is_theta_modulated, modulation_index, freqs, psd, lags, autocorr_full
    except Exception as e:
        is_theta_modulated=False
        modulation_index=0
        freqs=0
        psd=0
        lags=0
        autocorr_full=0

def plot_acg_and_psd(lags, autocorr, freqs, psd, theta_range=(6, 12)):
    """
    Plot the autocorrelogram (ACG) and power spectral density (PSD) with theta range highlighted.
    """
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # Plot ACG
    ax1.plot(lags, autocorr, 'b-', label='Autocorrelogram')
    ax1.set_xlabel('Time Lag (s)')
    ax1.set_ylabel('Correlation')
    ax1.set_title('Spike Train Autocorrelogram (Zero-lag Excluded)')
    ax1.grid(True)
    ax1.set_xlim(-0.5, 0.5)  # Limit to ±0.5 seconds for better visualization
    ax1.legend()
    
    # Plot PSD
    ax2.plot(freqs, psd, 'b-', label='PSD')
    ax2.axvspan(theta_range[0], theta_range[1], color='gray', alpha=0.2, label='Theta range (6-12 Hz)')
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Power Spectral Density')
    ax2.set_title('Power Spectral Density')
    ax2.grid(True)
    ax2.set_xlim(0, 50)  # Limit to 50 Hz for better visualization of theta range
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Example usage with synthetic spike times

# #spikes_time = np.sort(np.random.uniform(0, 10, 100))  # 100 random spikes over 10 seconds
# is_modulated, mod_index, freqs, psd, lags, autocorr = calculate_spike_theta_modulation(spikes_time)

# print(f"Theta modulation index: {mod_index:.2f}")
# print(f"Is theta modulated: {is_modulated}")

# # Plot both ACG and PSD
# plot_acg_and_psd(lags, autocorr, freqs, psd)


In [ ]:
base_folser = r"S:\Sachuriga\nwb\test4neo"
temp_nwb = pd.read_pickle(fr"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements_with_stability.pkl")
nwb_files = np.unique(temp_nwb["session_id"])
all_temp_nwb = []
temp_nwb["PS_p1"] = None
temp_nwb["spike_phase"] = None
temp_nwb["PS_cr1"] = None
temp_nwb["PS_slope1"] = None
temp_nwb["PS_phi1"] = None
temp_nwb["r1"] = None
temp_nwb["PS_p2"] = None
temp_nwb["PS_cr2"] = None
temp_nwb["PS_slope2"] = None
temp_nwb["PS_phi2"] = None
temp_nwb["r2"] = None
temp_nwb[ "PS_p3"] = None
temp_nwb[ "PS_cr3"] = None
temp_nwb["PS_slope3"] = None
temp_nwb["PS_phi3"] = None
temp_nwb["r3"] = None
temp_nwb["prefered_phase"] = None
temp_nwb["min_phase"] = None
temp_nwb["min_phase"] = None
temp_nwb["theta_index"] = None
temp_nwb["prefered_phase_smooth"] = None
temp_nwb["theta_caca"] = None

whole_phase=[]

In [ ]:
for file in nwb_files:
    
    pd_temp = temp_nwb[temp_nwb["session_id"]==file]
    unit_nums = pd_temp['unit_name']
    npdata = nap.load_file(fr"{base_folser}/{file}")
    position_cord = load_speed_fromNWB(npdata['XY_mid_brain'])
    lfp_times = npdata['lfp_times']
    lfp = npdata['lfp_raw']
    ## filter speed
    raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(position_cord[:,0], # times
                                position_cord[:,1], # x
                                position_cord[:,2], # y
                                filter_speed=True, 
                                min_speed = 0.1)
    units=npdata['units']
    spike_phase_couple=[]
    p1 = []
    p2 = []
    p3 = []
    cr1 = []
    cr2 = []
    cr3 = []
    r1=[]
    r2=[]
    r3=[]
    slope1 = []
    slope2 = []
    slope3 = []

    phi1 = []
    phi2 = []
    phi3 = []

    prefered_phase = []
    firing_phase = []
    min_phase = []
    theta_index = []  
    prefered_phase_smooth = []
    theta_caca=[]

    for unit_num in unit_nums:
        #unit_num = unit_nums[82]
        pd.set_option('display.max_rows', None)
        np.set_printoptions(threshold=np.inf)

        ## filter spikes with speed
        raw_pos = combined_array
        # ## filter spikes with speed
        # spk = speed_filtered_spikes(spikes_time,
        #                             pos_cord[:,0], # times
        #                             mask)
        #for i in range(40):
        # unit_idx = units[units['unit_name']==str(unit_num)].index[0]
        
        # spikes_time = np.array(units[unit_idx].as_series().index)
        spikes_time =pd_temp.loc[(pd_temp["session_id"] == file)& (pd_temp["unit_name"] == unit_num), "spike_times"].values[0]

        spikes_time = Speed_filtered_spikes(spikes_time,
                            position_cord[:,0],mask)
        #temp_caca = ThetaModulationIndex(spikes_time)
        spk = speed_filtered_spikes(spikes_time,
                                    position_cord[:,0])

        is_modulated, mod_index, freqs, psd, lags, autocorr = calculate_spike_theta_modulation(spikes_time)
        theta_index.append(mod_index)
        #theta_caca.append(temp_caca)

        # x_input = npdata['units']['x'][unit_idx]
        # y_input = npdata['units']['y'][unit_idx]

        #ch_num = unit_location_ch(x=x_input,y=y_input)
        ch_num = pd_temp.loc[(pd_temp["session_id"] == file)& (pd_temp["unit_name"] == unit_num), "ripple_ch_3std"].values[0]
        ## get rate_maps
        maps = mapp.SpatialMap(box_size=[1.0, 1.0], 
                            bin_size=0.05,
                            smoothing=0.1)

        ## generate ratemaps
        rate_map = maps.rate_map(raw_pos[:,1], 
                                raw_pos[:,2], 
                                raw_pos[:,0], 
                                spikes_time)

        ## get_fileds with center of the files
        fields = separate_fields_by_laplace_of_gaussian(rate_map, 
                                            minimum_field_area=9)
        fiesld_afremoval = remove_fields_by_area(fields, minimum_field_area=9)
        bc = calculate_field_centers(rate_map, fiesld_afremoval, center_method='center_of_mass')
        v = get_filed_num(fiesld_afremoval)
        original_field = fiesld_afremoval.copy()
        filed_size = []
        filde_name = []
        y_c=[]
        x_c=[]
        field_name = v[0]
        each_field = []
        ## get the center of the fields
        for field_nums in v:
            y_c.append(bc[field_nums-1][0])
            x_c.append(bc[field_nums-1][1])
            filed_size.append(len(fiesld_afremoval[fiesld_afremoval==field_nums]))
            filde_name.append(field_nums)
            current_field = original_field.copy()
            current_field[current_field != field_nums] = 0
            each_field.append(current_field)

        number = 0
        ## crosssings
        f = which_field(raw_pos[:,1],raw_pos[:,2],each_field[number],[1.0,1.0])
        in_field = np.array(np.where(f==filde_name[number]))
        indices = np.zeros(len(raw_pos[:,1]))
        indices[in_field]=1
        en,ex = compute_crossings(indices)
        filter_loger_runs = ex-en
        filtered_enter = en[filter_loger_runs>50]
        filtered_exit = ex[filter_loger_runs>50]

        restrcir_en=[]
        restrcir_ex=[]

        for i in range(len(filtered_enter)):
            if np.mean(smoothed_speed[filtered_enter[i]:filtered_exit[i]]) >= 0.05:
                restrcir_en.append(filtered_enter[i])
                restrcir_ex.append(filtered_exit[i])

        ## infields cords
        pos_cord=raw_pos
        xf = pos_cord[:,1][in_field]
        yf = pos_cord[:,2][in_field]
        tf = pos_cord[:,0][in_field]#
        spk_in = spk[in_field-1]

        x = pos_cord[:,1]
        y = pos_cord[:,2]
        t = pos_cord[:,0]

        full_phase=[]
        full_x=[]
        full_r=[]
        full_theta = []

        fs = 1000
        # 1. 带通滤波 (6-11 Hz)
        lfp_filtered = bandpass_filter(lfp[:,ch_num], 5, 12, fs)
        # 2. 使用希尔伯特变换计算瞬时相位
        analytic_signal = hilbert(lfp_filtered)
        # 计算相位（弧度）
        phase_rad = np.angle(analytic_signal)
        # 展开相位，避免-π到π的跳跃
        phase_rad_unwrapped = np.unwrap(phase_rad)
        # 转换为角度
        phase_deg = phase_rad_unwrapped * (180 / np.pi)
        theta_phase = phase_deg % 360 # 取模720，或者根据需求自定义

        for slide_s,slide_stop in zip(restrcir_en, restrcir_ex):
            #plot_path(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop],box_size=1,spike_times = spikes_time, ax=ax3)
            r, theta, pdcd, pdmd = map_pass_to_unit_circle(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop], x_c[number]/20, y_c[number]/20, field=each_field[number], box_size=[1.0, 1.0])
            phase_value=[]

            temp_time = t[slide_s:slide_stop]
            temp = spikes_time[spikes_time>t[slide_s]]
            temp1 = temp[temp<=t[slide_stop-1]]
            phase_value=[]
            phase_temp = []
            pdcd_temp = []
            r_temp = []
            
            if len(temp1)>0:
                for t1 in temp1:
                    index_run = np.searchsorted(temp_time, t1)
                    index_theta = np.searchsorted(lfp_times, t1)
                    phase_value.append(theta_phase[index_theta])
                    phase_temp.append(theta[index_run])
                    r_temp.append(r[index_run])
                    pdcd_temp.append(pdcd[index_run])

                    full_r.append(r[index_run])
                    full_phase.append(theta_phase[index_theta])
                    full_x.append(pdcd[index_run])
                    full_theta.append(theta[index_run])

        # Convert the list to a NumPy array for element-wise operations
        spike_phase = np.array(full_phase)

        bins = 360  # 分成 20 个区间，每区间 18 度
        hist, bin_edges = np.histogram(spike_phase, bins=bins, range=(0, 360))
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 每个区间的中心点

        # Step 3: 应用高斯平滑，sigma = 45 度
        # 因为 bin 的宽度是 18 度，sigma = 45 度需要换算成 bin 单位
        sigma_in_bins = 45/ (360 / bins)  # 45 度除以每个 bin 的宽度
        smoothed_hist = gaussian_filter1d(hist, sigma=sigma_in_bins, mode='wrap')  # 'wrap' 处理周期性边界
        max_value = np.max(smoothed_hist)
        peak_indices = np.where(smoothed_hist == max_value)[0]  # Find all indices with the max value
        peak_phases = bin_centers[peak_indices]  # Corresponding phase values
        preferred_phase = np.mean(peak_phases)

        prefered_phase.append(preferred_phase)
        firing_phase.append(np.max(hist))


        print(fr"prefers {preferred_phase}")

        # 步骤 1：将尖峰分到1°的bin中
        bins = np.arange(0, 360, 1)  # 360个bin，从0°到359°
        hist, bin_edges = np.histogram(full_phase, bins=bins, range=(0, 360))

        sigma = 45  # 标准差45°
        hist_smoothed = gaussian_filter1d(hist, sigma=sigma)  # 平滑
        prefered_phase_smooth.append(bin_centers[np.argmax(smoothed_hist)])

        # 步骤 3：找到最小尖峰数的相位
        min_phase_idx = np.argmin(hist_smoothed)  # 最小值的索引
        phase_zero = bins[min_phase_idx]  # 定义为相位零点
        min_phase.append(phase_zero)

        # 可选：调整相位（如果需要）
        # 如果想把所有相位相对于 phase_zero 重新定义
        adjusted_phases = (spike_phase + phase_zero) % 360

        degrees_list = adjusted_phases
        radians_list = []
        full_x=np.array(full_x)

        for deg in degrees_list:
            radians_list.append(math.radians(deg))

        radians_list=[]
        full_x_stacked = np.hstack([full_x, full_x])
        full_adjusted_phases = np.hstack([adjusted_phases, adjusted_phases+360])
        for deg in adjusted_phases:
            radians_list.append(math.radians(deg))
            
        spike_phase_couple.append(radians_list)
        
        whole_phase.append(radians_list) 
        mask_pos = (full_x_stacked <= 1) & (full_x_stacked >= -1)

        #slope_range = np.tan(np.linspace(-0.0333, -0.00033, 1000))

        try:
            circ_lin_corr, p_value, best_slope, theta_0, max_R = cl_corr(full_x, np.array(radians_list), -0.5, -0.025, 0.05, 1000, return_pval=True)       
            # circ_lin_corr, pp1, slope11, phi11, RR = cl_corr(full_x, np.array(radians_list), -0.5, -0.05, 0.05, 1000, return_pval=True)  
   
            # best_slope, max_R, theta_0 = circular_linear_fit(full_x, np.array(radians_list), slope_range)  
            # p_value = permutation_test(full_x, np.array(radians_list), best_slope, max_R, n_permutations=1000)

            first_idx = (full_x <= 0) & (full_x >= -1)
            secound_idx = (full_x > 0) & (full_x <= 1)
            print(fr"p val:{p_value}")
            print(fr"slope:{best_slope}")
            print(fr"R:{max_R}")

            p1.append(p_value)
            cr1.append(circ_lin_corr)
            slope1.append(best_slope)
            phi1.append(theta_0)
            r1.append(max_R)
        except Exception as e:
            p1.append(np.nan)
            cr1.append(np.nan)
            slope1.append(np.nan)
            phi1.append(np.nan)
            r1.append(np.nan)
        try:
            circ_lin_corr, p_value, best_slope, theta_0, max_R = cl_corr(full_x[first_idx], np.array(radians_list)[first_idx], -0.5, -0.025, 0.05, 1000, return_pval=True)
            # best_slope, max_R, theta_0 = circular_linear_fit(full_x[first_idx], np.array(radians_list)[first_idx], slope_range)  
            # p_value = permutation_test(full_x[first_idx], np.array(radians_list)[first_idx], best_slope, max_R, n_permutations=1000)

            print(fr"p val:{p_value}")
            print(fr"slope:{best_slope}")
            print(fr"corr:{max_R}")
            p2.append(p_value)         # Appending results
            cr2.append(circ_lin_corr)
            slope2.append(best_slope)
            phi2.append(theta_0)
            r2.append(max_R)
        except Exception as e:
            p2.append(np.nan)         # Appending results
            cr2.append(np.nan)
            slope2.append(np.nan)
            phi2.append(np.nan)
            r2.append(max_R)
        
        try:
            circ_lin_corr, p_value, best_slope, theta_0, max_R = cl_corr(full_x[secound_idx], np.array(radians_list)[secound_idx], -0.5, -0.025, 0.05, 1000, return_pval=True)

            # best_slope, max_R, theta_0 = circular_linear_fit(full_x[secound_idx], np.array(radians_list)[secound_idx], slope_range)  
            # p_value = permutation_test(full_x[secound_idx], np.array(radians_list)[secound_idx], best_slope, max_R, n_permutations=1000)

            print(fr"p val:{p_value}")
            print(fr"slope:{best_slope}")
            print(fr"corr:{max_R}")
            p3.append(p_value)         # Appending results
            cr3.append(circ_lin_corr)
            slope3.append(best_slope)
            phi3.append(theta_0)
            r3.append(max_R)
        except Exception as e:

            p3.append(np.nan)         # Appending results
            cr3.append(np.nan)
            slope3.append(np.nan)
            phi3.append(np.nan)
            r3.append(max_R)
        

    # # Assigning results to DataFrame (Correct approach using .loc)
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_p1"] = p1
    # #temp_nwb.loc[temp_nwb["session_id"] == file, "spike_phase"] = pd.Series(spike_phase_couple)
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_cr1"] = cr1
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_slope1"] = slope1
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_phi1"] = phi1
    # temp_nwb.loc[temp_nwb["session_id"] == file, "r1"] = r1
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_p2"] = p2
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_cr2"] = cr2
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_slope2"] = slope2
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_phi2"] = phi2
    # temp_nwb.loc[temp_nwb["session_id"] == file, "r2"] = r2
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_p3"] = p3
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_cr3"] = cr3
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_slope3"] = slope3
    # temp_nwb.loc[temp_nwb["session_id"] == file, "PS_phi3"] = phi3
    # temp_nwb.loc[temp_nwb["session_id"] == file, "r3"] = r3
    # temp_nwb.loc[temp_nwb["session_id"] == file, "prefered_phase"] = prefered_phase
    # temp_nwb.loc[temp_nwb["session_id"] == file, "min_phase"] = min_phase
    # temp_nwb.loc[temp_nwb["session_id"] == file, "min_phase"] = firing_phase
    # temp_nwb.loc[temp_nwb["session_id"] == file, "theta_index"] = theta_index 
    # temp_nwb.loc[temp_nwb["session_id"] == file, "prefered_phase_smooth"] = prefered_phase_smooth

        # Assigning results to DataFrame (Correct approach using .loc)
    pd_temp ["PS_p1"] = p1
    #temp_nwb.loc[temp_nwb["session_id"] == file, "spike_phase"] = pd.Series(spike_phase_couple)
    pd_temp ["PS_cr1"] = cr1
    pd_temp ["PS_slope1"] = slope1
    pd_temp ["PS_phi1"] = phi1
    pd_temp ["r1"] = r1
    pd_temp ["PS_p2"] = p2
    pd_temp ["PS_cr2"] = cr2
    pd_temp ["PS_slope2"] = slope2
    pd_temp [ "PS_phi2"] = phi2
    pd_temp ["r2"] = r2
    pd_temp [ "PS_p3"] = p3
    pd_temp ["PS_cr3"] = cr3
    pd_temp ["PS_slope3"] = slope3
    pd_temp ["PS_phi3"] = phi3
    pd_temp ["r3"] = r3
    pd_temp ["prefered_phase"] = prefered_phase
    pd_temp ["min_phase"] = min_phase
    pd_temp [ "min_phase"] = firing_phase
    pd_temp ["theta_index"] = theta_index 
    pd_temp ["prefered_phase_smooth"] = prefered_phase_smooth
    #temp_nwb.loc[temp_nwb["session_id"] == file, "theta_caca"] = theta_caca
    all_temp_nwb.append(pd_temp)

# Concatenate all temp_nwb DataFrames into a single DataFrame
combined_temp_nwb = pd.concat(all_temp_nwb, ignore_index=True)

# Save the combined DataFrame to a pickle file
combined_temp_nwb.to_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements_pycirc.pkl")


In [ ]:
combined_temp_nwb.to_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements_pycirc.pkl")

In [ ]:
combined_temp_nwb["spike_phase"]=None
combined_temp_nwb["spike_phase"]=pd.Series(whole_phase)

In [ ]:
import numpy as np
from scipy.stats import circmean, circvar

def circular_linear_fit(theta, x, slope_range):
    """
    Perform circular-linear analysis to quantify theta phase precession.
    
    Parameters:
    - theta: Array of instantaneous theta phases (in radians) for each spike.
    - x: Array of spatial positions of the animal at spike times.
    - slope_range: Array of slopes to test (in cycles/cm).
    
    Returns:
    - best_slope: Slope that maximizes the resultant length (R).
    - max_R: Maximum resultant length indicating model fit.
    - theta_0: Phase offset for the best fit.
    """
    def calculate_residuals(theta, x, slope, theta_0):
        """Calculate angular residuals for a given slope and phase offset."""
        theta_pred = (theta_0 + 2 * np.pi * slope * x) % (2 * np.pi)  # Predicted phase
        residuals = (theta - theta_pred) % (2 * np.pi)
        residuals = np.where(residuals > np.pi, residuals - 2 * np.pi, residuals)  # Wrap to [-π, π]
        return residuals

    def resultant_length(residuals):
        """Calculate resultant length (R) of angular residuals."""
        cos_sum = np.sum(np.cos(residuals))
        sin_sum = np.sum(np.sin(residuals))
        R = np.sqrt(cos_sum**2 + sin_sum**2) / len(residuals)
        return R

    best_R = -np.inf
    best_slope = None
    best_theta_0 = None

    # Test each slope in the provided range
    for slope in slope_range:
        # Estimate theta_0 using circular mean of residuals
        theta_pred = (2 * np.pi * slope * x) % (2 * np.pi)
        residuals = (theta - theta_pred) % (2 * np.pi)
        theta_0 = circmean(residuals, high=2*np.pi, low=0)
        
        # Calculate residuals and resultant length
        residuals = calculate_residuals(theta, x, slope, theta_0)
        R = resultant_length(residuals)
        
        # Update best fit if R is larger
        if R > best_R:
            best_R = R
            best_slope = slope
            best_theta_0 = theta_0

    return best_slope, best_R, best_theta_0

def permutation_test(theta, x, best_slope, best_R, n_permutations=1000):
    """
    Perform permutation test to estimate statistical significance of the fit.
    
    Parameters:
    - theta: Array of theta phases.
    - x: Array of spatial positions.
    - best_slope: Slope from the original fit.
    - best_R: Resultant length from the original fit.
    - n_permutations: Number of permutations for the test.
    
    Returns:
    - p_value: P-value indicating significance of the fit.
    """
    perm_R = []
    for _ in range(n_permutations):
        # Randomly permute theta values
        theta_perm = np.random.permutation(theta)
        slope, R, _ = circular_linear_fit(theta_perm, x, [best_slope])
        perm_R.append(R)
    
    # Calculate p-value as proportion of permutations with R >= best_R
    p_value = np.sum(np.array(perm_R) >= best_R) / n_permutations
    return p_value

In [ ]:
file= nwb_files[74]

pd_temp = temp_nwb[temp_nwb["session_id"]==file]
unit_nums = pd_temp['unit_name']
npdata = nap.load_file(fr"{base_folser}/{file}")
position_cord = load_speed_fromNWB(npdata['XY_mid_brain'])
lfp_times = npdata['lfp_times']
lfp = npdata['lfp_raw']
## filter speed
raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(position_cord[:,0], # times
                            position_cord[:,1], # x
                            position_cord[:,2], # y
                            filter_speed=True, 
                            min_speed = 0.1)
units=npdata['units']

p1 = []
p2 = []
p3 = []
cr1 = []
cr2 = []
cr3 = []

slope1 = []
slope2 = []
slope3 = []

phi1 = []
phi2 = []
phi3 = []

prefered_phase = []
firing_phase = []
min_phase = []
theta_index = []  
prefered_phase_smooth = []

for unit_num in unit_nums:
    #unit_num = unit_nums[82]
    pd.set_option('display.max_rows', None)
    np.set_printoptions(threshold=np.inf)

    ## filter spikes with speed
    raw_pos = combined_array
    # ## filter spikes with speed
    # spk = speed_filtered_spikes(spikes_time,
    #                             pos_cord[:,0], # times
    #                             mask)
    #for i in range(40):
    unit_idx = units[units['unit_name']==str(unit_num)].index[0]
    spikes_time = np.array(units[unit_idx].as_series().index)
    spikes_time = Speed_filtered_spikes(spikes_time,
                        position_cord[:,0],mask)

    spk = speed_filtered_spikes(spikes_time,
                                position_cord[:,0])

    is_modulated, mod_index, freqs, psd, lags, autocorr = calculate_spike_theta_modulation(spikes_time)
    theta_index.append(mod_index)

    x_input = npdata['units']['x'][unit_idx]
    y_input = npdata['units']['y'][unit_idx]

    ch_num = unit_location_ch(x=x_input,y=y_input)
    ## get rate_maps
    maps = mapp.SpatialMap(box_size=[1.0, 1.0], 
                        bin_size=0.05,
                        smoothing=0.1)

    ## generate ratemaps
    rate_map = maps.rate_map(raw_pos[:,1], 
                            raw_pos[:,2], 
                            raw_pos[:,0], 
                            spikes_time)

    ## get_fileds with center of the files
    fields = separate_fields_by_laplace_of_gaussian(rate_map, 
                                        minimum_field_area=9)
    fiesld_afremoval = remove_fields_by_area(fields, minimum_field_area=9)
    bc = calculate_field_centers(rate_map, fiesld_afremoval, center_method='center_of_mass')
    v = get_filed_num(fiesld_afremoval)
    original_field = fiesld_afremoval.copy()
    filed_size = []
    filde_name = []
    y_c=[]
    x_c=[]
    field_name = v[0]
    each_field = []
    ## get the center of the fields
    for field_nums in v:
        y_c.append(bc[field_nums-1][0])
        x_c.append(bc[field_nums-1][1])
        filed_size.append(len(fiesld_afremoval[fiesld_afremoval==field_nums]))
        filde_name.append(field_nums)
        current_field = original_field.copy()
        current_field[current_field != field_nums] = 0
        each_field.append(current_field)

    number = 0
    ## crosssings
    f = which_field(raw_pos[:,1],raw_pos[:,2],each_field[number],[1.0,1.0])
    in_field = np.array(np.where(f==filde_name[number]))
    indices = np.zeros(len(raw_pos[:,1]))
    indices[in_field]=1
    en,ex = compute_crossings(indices)
    filter_loger_runs = ex-en
    filtered_enter = en[filter_loger_runs>50]
    filtered_exit = ex[filter_loger_runs>50]

    restrcir_en=[]
    restrcir_ex=[]

    for i in range(len(filtered_enter)):
        if np.mean(smoothed_speed[filtered_enter[i]:filtered_exit[i]]) >= 0.05:
            restrcir_en.append(filtered_enter[i])
            restrcir_ex.append(filtered_exit[i])

    ## infields cords
    pos_cord=raw_pos
    xf = pos_cord[:,1][in_field]
    yf = pos_cord[:,2][in_field]
    tf = pos_cord[:,0][in_field]#
    spk_in = spk[in_field-1]

    x = pos_cord[:,1]
    y = pos_cord[:,2]
    t = pos_cord[:,0]

    full_phase=[]
    full_x=[]
    full_r=[]
    full_theta = []

    fs = 1000
    # 1. 带通滤波 (6-11 Hz)
    lfp_filtered = bandpass_filter(lfp[:,ch_num], 5, 12, fs)
    # 2. 使用希尔伯特变换计算瞬时相位
    analytic_signal = hilbert(lfp_filtered)
    # 计算相位（弧度）
    phase_rad = np.angle(analytic_signal)
    # 展开相位，避免-π到π的跳跃
    phase_rad_unwrapped = np.unwrap(phase_rad)
    # 转换为角度
    phase_deg = phase_rad_unwrapped * (180 / np.pi)
    theta_phase = phase_deg % 360 # 取模720，或者根据需求自定义

    for slide_s,slide_stop in zip(restrcir_en, restrcir_ex):
        #plot_path(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop],box_size=1,spike_times = spikes_time, ax=ax3)
        r, theta, pdcd, pdmd = map_pass_to_unit_circle(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop], x_c[number]/20, y_c[number]/20, field=each_field[number], box_size=[1.0, 1.0])
        phase_value=[]

        temp_time = t[slide_s:slide_stop]
        temp = spikes_time[spikes_time>t[slide_s]]
        temp1 = temp[temp<=t[slide_stop-1]]
        phase_value=[]
        phase_temp = []
        pdcd_temp = []
        r_temp = []
        
        if len(temp1)>0:
            for t1 in temp1:
                index_run = np.searchsorted(temp_time, t1)
                index_theta = np.searchsorted(lfp_times, t1)
                phase_value.append(theta_phase[index_theta])
                phase_temp.append(theta[index_run])
                r_temp.append(r[index_run])
                pdcd_temp.append(pdcd[index_run])

                full_r.append(r[index_run])
                full_phase.append(theta_phase[index_theta])
                full_x.append(pdcd[index_run])
                full_theta.append(theta[index_run])

    # Convert the list to a NumPy array for element-wise operations
    spike_phase = np.array(full_phase)

    bins = 360  # 分成 20 个区间，每区间 18 度
    hist, bin_edges = np.histogram(spike_phase, bins=bins, range=(0, 360))
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 每个区间的中心点

    # Step 3: 应用高斯平滑，sigma = 45 度
    # 因为 bin 的宽度是 18 度，sigma = 45 度需要换算成 bin 单位
    sigma_in_bins = 5/ (360 / bins)  # 45 度除以每个 bin 的宽度
    smoothed_hist = gaussian_filter1d(hist, sigma=sigma_in_bins, mode='wrap')  # 'wrap' 处理周期性边界
    max_value = np.max(smoothed_hist)
    peak_indices = np.where(smoothed_hist == max_value)[0]  # Find all indices with the max value
    peak_phases = bin_centers[peak_indices]  # Corresponding phase values
    preferred_phase = np.mean(peak_phases)

    prefered_phase.append(preferred_phase)
    firing_phase.append(np.max(hist))


    print(fr"prefers {preferred_phase}")

    # 步骤 1：将尖峰分到1°的bin中
    bins = np.arange(0, 360, 1)  # 360个bin，从0°到359°
    hist, bin_edges = np.histogram(full_phase, bins=bins, range=(0, 360))

    sigma = 45  # 标准差45°
    hist_smoothed = gaussian_filter1d(hist, sigma=sigma)  # 平滑
    prefered_phase_smooth.append(bin_centers[np.argmax(smoothed_hist)])

    # 步骤 3：找到最小尖峰数的相位
    min_phase_idx = np.argmin(hist_smoothed)  # 最小值的索引
    phase_zero = bins[min_phase_idx]  # 定义为相位零点
    min_phase.append(phase_zero)

    # 可选：调整相位（如果需要）
    # 如果想把所有相位相对于 phase_zero 重新定义
    adjusted_phases = (spike_phase + phase_zero) % 360

    degrees_list = adjusted_phases
    radians_list = []
    full_x=np.array(full_x)

    for deg in degrees_list:
        radians_list.append(math.radians(deg))

    radians_list=[]
    full_x_stacked = np.hstack([full_x, full_x])
    full_adjusted_phases = np.hstack([adjusted_phases, adjusted_phases+360])
    for deg in adjusted_phases:
        radians_list.append(math.radians(deg))
    mask_pos = (full_x_stacked <= 1) & (full_x_stacked >= -1)


    try:
        #circ_lin_corr, ci, slope, phi0, RR = cl_corr(full_x[mask], np.array(radians_list)[mask], -0.4, -0.001, 0.05, 1000, return_pval=True)       
        circ_lin_corr, pp1, slope11, phi11, RR = cl_corr(full_x, np.array(radians_list), -0.5, -0.01, 0.05, 1000, return_pval=True)       
        first_idx = (full_x <= 0) & (full_x >= -1)
        secound_idx = (full_x > 0) & (full_x <= 1)
        print(fr"p val:{pp1}")
        print(fr"slope:{slope11}")
        print(fr"corr:{circ_lin_corr}")
        p1.append(pp1)
        cr1.append(circ_lin_corr)
        slope1.append(slope11)
        phi1.append(phi11)
    except Exception as e:
        p1.append(np.nan)
        cr1.append(np.nan)
        slope1.append(np.nan)
        phi1.append(np.nan)
    try:
        circ_lin_corr, pp2, slope22, phi22, RR = cl_corr(full_x[first_idx], np.array(radians_list)[first_idx], -0.5, -0.01, 0.05, 1000, return_pval=True)
        print(fr"p val:{pp2}")
        print(fr"slope:{slope22}")
        print(fr"corr:{circ_lin_corr}")
        p2.append(pp2)         # Appending results
        cr2.append(circ_lin_corr)
        slope2.append(slope22)
        phi2.append(phi22)
    except Exception as e:
        p2.append(np.nan)         # Appending results
        cr2.append(np.nan)
        slope2.append(np.nan)
        phi2.append(np.nan)
    
    try:
        circ_lin_corr, pp3, slope33, phi33, RR = cl_corr(full_x[secound_idx], np.array(radians_list)[secound_idx], -0.5, -0.01, 0.05, 1000, return_pval=True)
        print(fr"p val:{pp3}")
        print(fr"slope:{slope33}")
        print(fr"corr:{circ_lin_corr}")
        p3.append(pp3)         # Appending results
        cr3.append(circ_lin_corr)
        slope3.append(slope33)
        phi3.append(phi33)
    except Exception as e:

        p3.append(np.nan)         # Appending results
        cr3.append(np.nan)
        slope3.append(np.nan)
        phi3.append(np.nan)

# Assigning results to DataFrame (Correct approach using .loc)
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_p1"] = p1
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_cr1"] = cr1
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_slope1"] = slope1
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_phi1"] = phi1

temp_nwb.loc[temp_nwb["session_id"] == file, "PS_p2"] = p2
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_cr2"] = cr2
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_slope2"] = slope2
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_phi2"] = phi2

temp_nwb.loc[temp_nwb["session_id"] == file, "PS_p3"] = p3
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_cr3"] = cr3
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_slope3"] = slope3
temp_nwb.loc[temp_nwb["session_id"] == file, "PS_phi3"] = phi3

temp_nwb.loc[temp_nwb["session_id"] == file, "prefered_phase"] = prefered_phase
temp_nwb.loc[temp_nwb["session_id"] == file, "min_phase"] = min_phase
temp_nwb.loc[temp_nwb["session_id"] == file, "min_phase"] = firing_phase
temp_nwb.loc[temp_nwb["session_id"] == file, "theta_index"] = theta_index 
temp_nwb.loc[temp_nwb["session_id"] == file, "prefered_phase_smooth"] = prefered_phase_smooth


In [ ]:
temp_nwb.loc[temp_nwb["session_id"] == file, ["cell_type","theta_index", "PS_p1", "PS_slope1"]] 